**ANN Project**

In [2]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2025-04-29 10:17:13.565747: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prepping the dataset for training the model

#### Cleaning milage, price, and dropping some columns

In [3]:
# Load the dataset
used_cars = pd.read_csv('used_cars.csv')

used_cars = used_cars.drop(columns=['int_col', 'clean_title'])

print("Before dropping rows with NaN values:")
print(used_cars.shape[0])

used_cars = used_cars.dropna()

print("After dropping rows with NaN values:")
print(used_cars.shape[0])

used_cars['price'] = used_cars['price'].str.replace('$', '').str.replace(',', '').astype(int)
used_cars['milage'] = used_cars['milage'].str.replace('mi.', '').str.replace(',', '').astype(int)

used_cars.head(10)

Before dropping rows with NaN values:
4009
After dropping rows with NaN values:
3730


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,At least 1 accident or damage reported,10300
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,At least 1 accident or damage reported,38005
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,None reported,54598
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,None reported,15500
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,None reported,34999
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,F,Silver,None reported,14798
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,None reported,31000
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,None reported,7300
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,None reported,41927
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Automatic,Fuji White,None reported,73897


#### Cleaning the Accident Column (this can be kept as it is. no need for one hot encoding)

In [4]:
# print(used_cars['accident'].unique())

def clean_accident(text):
    if pd.isna(text):
        return None
    elif "None reported" in text:
        return 0
    else:
        return 1
    
used_cars["accident_reported"] = used_cars["accident"].apply(clean_accident)
used_cars = used_cars.dropna(subset=["accident_reported"])

# print(used_cars['accident_reported'].value_counts(dropna=False))

print(used_cars.head(100).to_string())

             brand                                         model  model_year  milage       fuel_type                                                                 engine                       transmission                             ext_col                                accident   price  accident_reported
0             Ford               Utility Police Interceptor Base        2013   51000   E85 Flex Fuel                   300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability                        6-Speed A/T                               Black  At least 1 accident or damage reported   10300                  1
1          Hyundai                                  Palisade SEL        2021   34742        Gasoline                                                   3.8L V6 24V GDI DOHC                  8-Speed Automatic                     Moonlight Cloud  At least 1 accident or damage reported   38005                  1
2            Lexus                                 RX 350 RX 350      

#### Cleaning & One-Hot Encoding Transmission

Here we are making the assumption that for example 7-speed and 6-speed a/t can be grouped together. to prove our logic, we might need the map that moussa mentioned in our meeting. 

#### HERE ADD THE MAP THAT MOUSSA MENTIONED IN THE MEETING (ABOUT HOW HE PROVES CATTLES AND ANOTHER ANIMALS CAN BE GROUPED INTO THE SAME CATEGORY)

In [54]:
# THE CODE FOR THE MAP

In [5]:
def clean_transmission(text):
    if pd.isna(text):
        return None
    text = text.lower()
    if 'automatic' in text or 'a/t' in text:
        return "automatic"
    elif 'manual' in text or 'm/t' in text:
        return "manual"
    elif 'dual' in text or 'shift' in text or 'dsg' in text:
        return "dual_shift"
    elif 'cvt' in text:
        return "cvt"
    elif 'semi-automatic' in text or 'semi automatic' in text:
        return "semi_automatic"
    else:
        return "unknown"

# Apply the function to clean the transmission column
used_cars["transmission"] = used_cars["transmission"].apply(clean_transmission)

# Drop rows with NaN values in the 'transmission' column
used_cars = used_cars.dropna(subset=["transmission"])

# One-hot encode the cleaned 'transmission' column
used_cars = pd.get_dummies(used_cars, columns=["transmission"], prefix="trans")
# Convert boolean columns to integers (0/1)
used_cars.loc[:, used_cars.columns.str.startswith("trans_")] = \
    used_cars.loc[:, used_cars.columns.str.startswith("trans_")].astype(int)


used_cars.head(50)

/var/folders/xj/xrsyxvbd1yq0781nfdl0dkzr0000gn/T/ipykernel_14983/3804854250.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  used_cars.loc[:, used_cars.columns.str.startswith("trans_")] = \
/var/folders/xj/xrsyxvbd1yq0781nfdl0dkzr0000gn/T/ipykernel_14983/3804854250.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  used_cars.loc[:, used_cars.columns.str.startswith("trans_")] = \
/var/folders/xj/xrsyxvbd1yq0781nfdl0dkzr0000gn/T/ipykernel_14983/3804854250.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 1 0 0]' has dtype incompatible with bool, 

,brand,model,model_year,milage,fuel_type,engine,ext_col,accident,price,accident_reported,trans_automatic,trans_cvt,trans_dual_shift,trans_manual,trans_unknown
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,Black,At least 1 accident or damage reported,10300,1,1,0,0,0,0
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,Moonlight Cloud,At least 1 accident or damage reported,38005,1,1,0,0,0,0
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Blue,None reported,54598,0,1,0,0,0,0
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,Black,None reported,15500,0,1,0,0,0,0
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,Glacier White Metallic,None reported,34999,0,1,0,0,0,0
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,Silver,None reported,14798,0,0,0,0,0,1
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Blue,None reported,31000,0,1,0,0,0,0
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,Green,None reported,7300,0,1,0,0,0,0
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,Black,None reported,41927,0,1,0,0,0,0
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Fuji White,None reported,73897,0,1,0,0,0,0


#### Cleaning and One-Hot Encoding Brand

In [6]:
# One-hot encode the 'brand' column
used_cars = pd.get_dummies(used_cars, columns=['brand'], prefix='brand')

# Convert any boolean columns created to integers (0/1)
brand_cols = used_cars.columns[used_cars.columns.str.startswith('brand_')]
used_cars[brand_cols] = used_cars[brand_cols].astype(int)

used_cars.head(10)

,model,model_year,milage,fuel_type,engine,ext_col,accident,price,accident_reported,trans_automatic,...,brand_Rolls-Royce,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Toyota,brand_Volkswagen,brand_Volvo,brand_smart
0,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,Black,At least 1 accident or damage reported,10300,1,1,...,0,0,0,0,0,0,0,0,0,0
1,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,Moonlight Cloud,At least 1 accident or damage reported,38005,1,1,...,0,0,0,0,0,0,0,0,0,0
2,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Blue,None reported,54598,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,Black,None reported,15500,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,Glacier White Metallic,None reported,34999,0,1,...,0,0,0,0,0,0,0,0,0,0
5,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,Silver,None reported,14798,0,0,...,0,0,0,0,0,0,0,0,0,0
6,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Blue,None reported,31000,0,1,...,0,0,0,0,0,0,0,0,0,0
7,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,Green,None reported,7300,0,1,...,0,0,0,0,0,0,0,0,0,0
8,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,Black,None reported,41927,0,1,...,0,0,0,0,0,0,0,0,0,0
10,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Fuji White,None reported,73897,0,1,...,0,0,0,0,0,0,0,0,0,0


#### Cleaning and One-Hot Encoding ext_col

### WE ALSO NEED THE MAP THAT MOUSSA MENTIONED HERE TO PROVE THAT OUR GROUPING MAKES SENSE! also we need to do something with the non-standard colors. we should probably not classify them as one category. the colors that fall into that category might be very different from one another. 

In [57]:
# the code for the moussa map

In [7]:
# Clean and simplify the color column
def simplify_color(color):
    color = str(color).lower()

    if any(x in color for x in ['white', 'chalk', 'pearl', 'bianco']):
        return 'White'
    elif any(x in color for x in ['black', 'nero', 'ebony', 'obsidian']):
        return 'Black'
    elif any(x in color for x in ['blue', 'blu', 'stormy']):
        return 'Blue'
    elif any(x in color for x in ['gray', 'grey', 'graphite', 'slate', 'granite', 'quartzite']):
        return 'Gray'
    elif any(x in color for x in ['red', 'rosso', 'ruby', 'scarlet']):
        return 'Red'
    elif any(x in color for x in ['silver', 'iridium', 'tungsten']):
        return 'Silver'
    elif any(x in color for x in ['green', 'moss', 'verde']):
        return 'Green'
    elif any(x in color for x in ['brown', 'bronze', 'dune']):
        return 'Brown'
    elif any(x in color for x in ['orange', 'mango', 'arancio']):
        return 'Orange'
    elif any(x in color for x in ['yellow', 'hellayella']):
        return 'Yellow'
    elif any(x in color for x in ['purple', 'plum', 'ametrin']):
        return 'Purple'
    elif any(x in color for x in ['beige', 'tan', 'sandstone']):
        return 'Beige'
    elif any(x in color for x in ['gold']):
        return 'Gold'
    else:
        return 'Non-Standard Color'

# Apply function
used_cars['ext_color_simple'] = used_cars['ext_col'].apply(simplify_color)
used_cars = used_cars.dropna(subset=["ext_color_simple"])

# One-hot encode ext_color_simple
used_cars = pd.get_dummies(used_cars, columns=['ext_color_simple'], prefix='color')

# Convert boolean columns to int
color_cols = used_cars.columns[used_cars.columns.str.startswith('color_')]
used_cars[color_cols] = used_cars[color_cols].astype(int)

used_cars = used_cars.drop(columns=['ext_col'])

used_cars.head(10)

,model,model_year,milage,fuel_type,engine,accident,price,accident_reported,trans_automatic,trans_cvt,...,color_Gold,color_Gray,color_Green,color_Non-Standard Color,color_Orange,color_Purple,color_Red,color_Silver,color_White,color_Yellow
0,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,At least 1 accident or damage reported,10300,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,At least 1 accident or damage reported,38005,1,1,0,...,0,0,0,1,0,0,0,0,0,0
2,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,None reported,54598,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,None reported,15500,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,None reported,34999,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,None reported,14798,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,None reported,31000,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,None reported,7300,0,1,0,...,0,0,1,0,0,0,0,0,0,0
8,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,None reported,41927,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,None reported,73897,0,1,0,...,0,0,0,0,0,0,0,0,1,0


#### Cleaning and One Hot Encoding Fuel Type

### AGAIN WE MIGHT NEED THE MOUSSA MAP HERE. AND WE NEED TO FIX THE "NEEDS_WORK" CATEGORY BELOW, WHICH WAS PREVIOUSLY REFERRED AS 4

In [59]:
# code for the map

In [8]:
print(used_cars['fuel_type'].value_counts(dropna=False))

def clean_fuel(value):
    value = value.lower()
    if 'gasoline' in value:
        return "gasoline"
    elif 'diesel' in value:
        return "diesel"
    elif 'hybrid' in value:
        return "hybrid"
    elif 'e85 flex fuel' in value:
        return "e85 flex fuel"
    else:
        return "unknown"

used_cars['fuel_type'] = used_cars['fuel_type'].apply(clean_fuel)

used_cars = pd.get_dummies(used_cars, columns=['fuel_type'], prefix='fuel')
used_cars[used_cars.columns[used_cars.columns.str.startswith("fuel_")]] = \
    used_cars[used_cars.columns[used_cars.columns.str.startswith("fuel_")]].astype(int)


used_cars.head(10)

fuel_type
Gasoline          3218
Hybrid             190
E85 Flex Fuel      133
Diesel             110
–                   44
Plug-In Hybrid      33
not supported        2
Name: count, dtype: int64


,model,model_year,milage,engine,accident,price,accident_reported,trans_automatic,trans_cvt,trans_dual_shift,...,color_Purple,color_Red,color_Silver,color_White,color_Yellow,fuel_diesel,fuel_e85 flex fuel,fuel_gasoline,fuel_hybrid,fuel_unknown
0,Utility Police Interceptor Base,2013,51000,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,At least 1 accident or damage reported,10300,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Palisade SEL,2021,34742,3.8L V6 24V GDI DOHC,At least 1 accident or damage reported,38005,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,RX 350 RX 350,2022,22372,3.5 Liter DOHC,None reported,54598,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Q50 Hybrid Sport,2015,88900,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,None reported,15500,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Q3 45 S line Premium Plus,2021,9835,2.0L I4 16V GDI DOHC Turbo,None reported,34999,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
5,ILX 2.4L,2016,136397,2.4 Liter,None reported,14798,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
6,S3 2.0T Premium Plus,2017,84000,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,None reported,31000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
7,740 iL,2001,242000,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,None reported,7300,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
8,RC 350 F Sport,2021,23436,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,None reported,41927,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
10,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,V6,None reported,73897,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0


## Key for Feature Extraction from 'engine' Column
#### horsepower     → Extracted from patterns like '300.0HP'
#### engine_size    → Extracted from patterns like '3.7L' or '3.5 Liter'
#### cylinder_type  → Extracted from patterns like 'V6' or '4 Cylinder'

In [9]:
import re

# Extract horsepower (e.g., '300.0HP')
used_cars['horsepower'] = used_cars['engine'].apply(lambda x: re.findall(r'(\d+(\.\d+)?)HP', str(x)))
used_cars['horsepower'] = used_cars['horsepower'].apply(lambda x: x[0][0] if x else None)

# Extract engine size (e.g., '3.7L', '2.0L')
used_cars['engine_size'] = used_cars['engine'].apply(lambda x: re.findall(r'(\d+(\.\d+)?)L|(\d+(\.\d+)?) Liter', str(x)))
used_cars['engine_size'] = used_cars['engine_size'].apply(lambda x: x[0][0] if x else None)

# Extract cylinder type (e.g., 'V6', '4 Cylinder')
used_cars['cylinder_type'] = used_cars['engine'].apply(lambda x: re.findall(r'V\d|(\d+ Cylinder)', str(x)))
used_cars['cylinder_type'] = used_cars['cylinder_type'].apply(lambda x: x[0] if x else None)

# Count missing (None or NaN) values in the new columns
missing_horsepower = used_cars['horsepower'].isna().sum()
missing_engine_size = used_cars['engine_size'].isna().sum()
missing_cylinder_type = used_cars['cylinder_type'].isna().sum()

print("Missing values:")
print(f"Horsepower: {missing_horsepower}")
print(f"Engine Size: {missing_engine_size}")
print(f"Cylinder Type: {missing_cylinder_type}")

# Check how many rows have BOTH horsepower and engine_size missing
both_missing = used_cars[used_cars['horsepower'].isna() & used_cars['engine_size'].isna()]
print(f"Rows with BOTH horsepower and engine_size missing: {both_missing.shape[0]}")

missing_any = used_cars[['horsepower', 'engine_size', 'cylinder_type']].isnull().any(axis=1)
print("Rows with at least one missing engine-related value:", missing_any.sum())


used_cars = used_cars[~(
    used_cars['horsepower'].isnull() &
    used_cars['engine_size'].isnull() &
    used_cars['cylinder_type'].isnull()
)]

missing_count = used_cars[['horsepower', 'engine_size', 'cylinder_type']].isnull().sum(axis=1)

print("Exactly 1 missing:", (missing_count == 1).sum())
print("Exactly 2 missing:", (missing_count == 2).sum())
print("All 3 missing:", (missing_count == 3).sum())


used_cars.head(10) 

Missing values:
Horsepower: 746
Engine Size: 55
Cylinder Type: 441
Rows with BOTH horsepower and engine_size missing: 53
Rows with at least one missing engine-related value: 753
Exactly 1 missing: 313
Exactly 2 missing: 391
All 3 missing: 0


,model,model_year,milage,engine,accident,price,accident_reported,trans_automatic,trans_cvt,trans_dual_shift,...,color_White,color_Yellow,fuel_diesel,fuel_e85 flex fuel,fuel_gasoline,fuel_hybrid,fuel_unknown,horsepower,engine_size,cylinder_type
0,Utility Police Interceptor Base,2013,51000,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,At least 1 accident or damage reported,10300,1,1,0,0,...,0,0,0,1,0,0,0,300.0,3.7,
1,Palisade SEL,2021,34742,3.8L V6 24V GDI DOHC,At least 1 accident or damage reported,38005,1,1,0,0,...,0,0,0,0,1,0,0,None,3.8,
2,RX 350 RX 350,2022,22372,3.5 Liter DOHC,None reported,54598,0,1,0,0,...,0,0,0,0,1,0,0,None,,None
3,Q50 Hybrid Sport,2015,88900,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,None reported,15500,0,1,0,0,...,0,0,0,0,0,1,0,354.0,3.5,
4,Q3 45 S line Premium Plus,2021,9835,2.0L I4 16V GDI DOHC Turbo,None reported,34999,0,1,0,0,...,1,0,0,0,1,0,0,None,2.0,None
5,ILX 2.4L,2016,136397,2.4 Liter,None reported,14798,0,0,0,0,...,0,0,0,0,1,0,0,None,,None
6,S3 2.0T Premium Plus,2017,84000,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,None reported,31000,0,1,0,0,...,0,0,0,0,1,0,0,292.0,2.0,4 Cylinder
7,740 iL,2001,242000,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,None reported,7300,0,1,0,0,...,0,0,0,0,1,0,0,282.0,4.4,8 Cylinder
8,RC 350 F Sport,2021,23436,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,None reported,41927,0,1,0,0,...,0,0,0,0,1,0,0,311.0,3.5,
10,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,V6,None reported,73897,0,1,0,0,...,1,0,0,0,1,0,0,None,None,


In [62]:
# Displaying all columns
print(used_cars.columns.tolist())

['model', 'model_year', 'milage', 'engine', 'accident', 'price', 'accident_reported', 'trans_automatic', 'trans_cvt', 'trans_dual_shift', 'trans_manual', 'trans_unknown', 'brand_Acura', 'brand_Alfa', 'brand_Aston', 'brand_Audi', 'brand_BMW', 'brand_Bentley', 'brand_Bugatti', 'brand_Buick', 'brand_Cadillac', 'brand_Chevrolet', 'brand_Chrysler', 'brand_Dodge', 'brand_FIAT', 'brand_Ferrari', 'brand_Ford', 'brand_GMC', 'brand_Genesis', 'brand_Honda', 'brand_Hummer', 'brand_Hyundai', 'brand_INFINITI', 'brand_Jaguar', 'brand_Jeep', 'brand_Kia', 'brand_Lamborghini', 'brand_Land', 'brand_Lexus', 'brand_Lincoln', 'brand_Lotus', 'brand_MINI', 'brand_Maserati', 'brand_Maybach', 'brand_Mazda', 'brand_McLaren', 'brand_Mercedes-Benz', 'brand_Mercury', 'brand_Mitsubishi', 'brand_Nissan', 'brand_Plymouth', 'brand_Pontiac', 'brand_Porsche', 'brand_RAM', 'brand_Rolls-Royce', 'brand_Saab', 'brand_Saturn', 'brand_Scion', 'brand_Subaru', 'brand_Suzuki', 'brand_Toyota', 'brand_Volkswagen', 'brand_Volvo', 'b

In [10]:
# Columns to drop
drop_columns = ['model', 'engine', 'accident', 'horsepower', 'engine_size', 'cylinder_type', 'price']

# X will be inputs, y will be target
X = used_cars.drop(columns=drop_columns)
y = used_cars['price']

# Normalize model_year and milage
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X[['model_year', 'milage']] = scaler.fit_transform(X[['model_year', 'milage']])

print(X.shape, y.shape)

(3681, 79) (3681,)


In [ ]:
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Trial 1: Test MAE: 26243.37

In [ ]:
# Build the feedforward neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression (no activation or linear)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',    # Mean Squared Error
    metrics=['mae']  # Mean Absolute Error 
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[callback])

# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

# Make predictions (maybe)
# predictions = model.predict(X_test)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 5970605568.0000 - mae: 42963.4570 - val_loss: 4089363200.0000 - val_mae: 42251.9258
Epoch 2/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 5602045440.0000 - mae: 42930.4453 - val_loss: 4084267264.0000 - val_mae: 42190.2070
Epoch 3/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5754815488.0000 - mae: 41608.5859 - val_loss: 4068175872.0000 - val_mae: 41994.7070
Epoch 4/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6446596096.0000 - mae: 43711.8750 - val_loss: 4035376128.0000 - val_mae: 41592.9727
Epoch 5/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5596815872.0000 - mae: 41728.4492 - val_loss: 3981211648.0000 - val_mae: 40920.5039
Epoch 6/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9788725248.0000 - mae: 44007.1992 - val_loss: 3903233280.0000 - val_mae: 39933.9766
Epoch 7/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6381165056.0000 - mae: 40500.6836 - val_loss: 3799185664.0000 - val_mae: 38592.5703

In [12]:
used_cars['price'].describe()

count    3.681000e+03
mean     4.409164e+04
std      8.096105e+04
min      2.000000e+03
25%      1.699500e+04
50%      3.000000e+04
75%      4.888900e+04
max      2.954083e+06
Name: price, dtype: float64